In [10]:
#| include: false
from nbdev.showdoc import *

This part of the tutorial demonstrates how to use `NumerFrame` to download predictions bought on [NumerBay](http://numerbay.ai/) community marketplace. Currently only the main tournament is supported. Signals support will be added in future.

In [11]:
from nbdev import show_doc
from numerblox.numerframe import create_numerframe, NumerFrame
from numerblox.download import NumeraiClassicDownloader
from numerblox.key import Key
from numerblox.model import NumerBayCSVs
from numerblox.submission import NumerBaySubmitter, NumeraiClassicSubmitter, NumeraiSignalsSubmitter

First, we download tournament data using `NumeraiClassicDownloader`.

In [12]:
downloader = NumeraiClassicDownloader("numerframe_edu")
# Path variables
tournament_file = "numerai_tournament_data.parquet"
tournament_save_path = f"{str(downloader.dir)}/{tournament_file}"
# Download only tournament parquet file
downloader.download_single_dataset(tournament_file,
                                   dest_path=tournament_save_path)

📁 Downloading 'numerai_tournament_data.parquet' 📁

2023-01-04 20:22:00,026 INFO numerapi.utils: target file already exists
2023-01-04 20:22:00,028 INFO numerapi.utils: download complete


Loading in data and initializing a `NumerFrame` takes one line of code. It will automatically recognize the data format such as `.csv` or `.parquet`. You have the option to add metadata, which is stored in the `meta` attribute.

In [13]:
# Initialize NumerFrame from parquet file path
dataf = create_numerframe(tournament_save_path, metadata={"version": 3, "type": "tournament"})

In [14]:
dataf.head(2)

,era,data_type,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,feature_demisable_expiring_millepede,...,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,0575,test,0.50,0.0,1.0,0.0,0.75,0.00,1.00,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
n001e1318d5072ac,0575,test,0.25,1.0,0.5,0.5,0.25,0.75,0.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, we create a NumerBayCSVs model object with a list of products we bought and NumerBay credentials. `numerbay_key_path` may be required for products that use client-side encryption. You can learn more about encryption [here](https://docs.numerbay.ai/updates/encryption)

In [15]:
nb_model = NumerBayCSVs(data_directory='/app/notebooks/tmp',
                        numerbay_product_full_names=['numerai-predictions-numerbay'],  # change to the full names of products you bought
                        numerbay_username="your_username",  # change to your own username
                        numerbay_password="your_password",  # change to your own password
                        numerbay_key_path="/app/notebooks/tmp/numerbay.json")

Call the `predict` method on the `NumerFrame` to fetch the prediction file from NumerBay. If the file already exists in the `data_directory`, that file will be loaded without re-downloading.

In [16]:
# preds = nb_model.predict(dataf)

The predictions are concatenated to the `NumerFrame` with column name `prediction_numerai-predictions-numerbay`

In [17]:
# preds

In this part of the tutorial we have downloaded a prediction file from NumerBay with `NumerFrame`. This makes things easier for post processing such as ensembling and neutralization.

------------------------------------------------------

# NumerBay submission

This part of the tutorial is for sellers who want to upload their predictions to NumerBay to fulfill sale orders. Using `NumerBaySubmitter`, a seller can choose to submit to both Numerai and NumerBay or just NumerBay.

Assume we have some prediction column to upload for the Numerai main tournament, in this case the `prediction` column which simply takes the value of a feature.

In [18]:
# dataf = create_numerframe(tournament_save_path)

In [19]:
# dataf['prediction'] = dataf['feature_dichasial_hammier_spawner']

Set `upload_to_numerai` to True (default) if you want to submit to both Numerai and NumerBay, set to False to submit only to NumerBay.

In [20]:
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=True, numerbay_username="numerbay", numerbay_password="your_password")

Finally, we call the `full_submission` method to perform the submission

In [21]:
# nb_submitter.full_submission(dataf, file_name='upload-full.csv', model_name='numerbay', numerbay_product_full_name='numerai-predictions-numerbay', cols='prediction')

The process for Signals submission is very similar and is omitted for brevity, just do the following:
- Use Signals NumerFrame
- Change `NumeraiClassicSubmitter` to `NumeraiSignalsSubmitter` for the `tournament_submitter` argument
- When calling `full_submission`, change the `cols` argument to the list of Signals column to submit (e.g. `['bloomberg_ticker', 'signal']`)

------------------------------------------------------

After we are done we can easily clean up our downloaded data with one line of code called from the downloader.

In [22]:
# Clean up environment
downloader.remove_base_directory()

⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: '/home/clepelaars/numerblox/nbs/edu_nbs/numerframe_edu'